In [8]:
from pymongo import MongoClient
from dotenv import load_dotenv
import os

load_dotenv()

SSH_HOST = os.getenv("SSH_HOST")
SSH_PORT = int(os.getenv("SSH_PORT", 22))
SSH_USER = os.getenv("SSH_USER")
SSH_KEY = os.getenv("SSH_KEY")

# MongoDB 설정
MONGO_HOST = os.getenv("MONGO_HOST", "localhost")
MONGO_PORT = int(os.getenv("MONGO_PORT", 27017))
MONGO_USER = os.getenv("MONGO_USER")
MONGO_PASSWORD = os.getenv("MONGO_PASSWORD")
MONGO_AUTH_DB = os.getenv("MONGO_AUTH_DB", "admin")

import warnings
warnings.filterwarnings("ignore", module="paramiko")
from sshtunnel import SSHTunnelForwarder


server = SSHTunnelForwarder(
    (SSH_HOST, SSH_PORT),
    ssh_username=SSH_USER,
    ssh_pkey=SSH_KEY,
    remote_bind_address=(MONGO_HOST, MONGO_PORT)
)
server.start()

client = MongoClient(
    f"mongodb://{MONGO_USER}:{MONGO_PASSWORD}"
    f"@127.0.0.1:{server.local_bind_port}/?authSource={MONGO_AUTH_DB}"
    )

print(client.list_database_names())


['class']


In [4]:
client.list_database_names()

['class']

In [9]:
db = client["class"]
print(db.list_collection_names())
students = db["students"]
dummy = db['dummy']

['dummy', 'students', 'student', 'courses']


In [16]:
doc = {"name": "Alice", "age": 22}
dummy.insert_one(doc)

InsertOneResult(ObjectId('692d87c91c54f1da6b6a932c'), acknowledged=True)

In [17]:
students.insert_many([
    {"name": "Bob", "age": 23},
    {"name": "Charlie", "age": 24},
    {"name": "Jihoo", "age": 13}
])

InsertManyResult([ObjectId('692d87ca1c54f1da6b6a932d'), ObjectId('692d87ca1c54f1da6b6a932e'), ObjectId('692d87ca1c54f1da6b6a932f')], acknowledged=True)

In [18]:
doc = students.find_one({"name": "Alice"})
print(doc)

None


In [19]:
docs = list(students.find())
print(docs)

[{'_id': ObjectId('692d87ca1c54f1da6b6a932d'), 'name': 'Bob', 'age': 23}, {'_id': ObjectId('692d87ca1c54f1da6b6a932e'), 'name': 'Charlie', 'age': 24}, {'_id': ObjectId('692d87ca1c54f1da6b6a932f'), 'name': 'Jihoo', 'age': 13}]


In [ ]:
'''
조건 검색: 기본 비교 연산자 > < >= <= !=
'''

# age > 22
cursor = list(students.find({
    "age": {
        "$gt": 22
    }
})) # gt: greater than
print(cursor)

# age >= 22 and age <= 30 (범위)
cursor = list(students.find({
    "age": {"$gte": 22, "$lte": 30} # gte: greater than or equal to (>=), lte: less than or equal to (<=)
}))
print(cursor)

# age != 22
cursor = list(students.find({
    "age": {"$ne": 22}
})) # ne: not equal to
print(cursor)


[{'_id': ObjectId('692d87ca1c54f1da6b6a932d'), 'name': 'Bob', 'age': 23}, {'_id': ObjectId('692d87ca1c54f1da6b6a932e'), 'name': 'Charlie', 'age': 24}]
[{'_id': ObjectId('692d87ca1c54f1da6b6a932d'), 'name': 'Bob', 'age': 23}, {'_id': ObjectId('692d87ca1c54f1da6b6a932e'), 'name': 'Charlie', 'age': 24}]
[{'_id': ObjectId('692d87ca1c54f1da6b6a932d'), 'name': 'Bob', 'age': 23}, {'_id': ObjectId('692d87ca1c54f1da6b6a932e'), 'name': 'Charlie', 'age': 24}, {'_id': ObjectId('692d87ca1c54f1da6b6a932f'), 'name': 'Jihoo', 'age': 13}]


In [9]:
'''
조건 검색: 포함/배제
'''

# name이 Alice 또는 Bob인 문서
students.find({"name": {"$in": ["Alice", "Bob"]}})

# 특정 값들을 제외
students.find({"name": {"$nin": ["Eve", "Mallory"]}})

In [ ]:
'''
조건 검색: 논리 연산자
'''

# 암묵적 AND (같은 수준의 필드 여러 조건) 
# --> and를 따로 지정 안 해도 and로 처리
students.find({
    "age": {"$gt": 18}, 
    "grade": "A"
})

# OR
students.find({
    "$or": [
        {"age": {"$lt": 18}},
        {"grade": "A"}
    ]
})

# AND (명시적)
students.find({
    "$and": [
        {"age": {"$gt": 18}}, 
        {"grade": "B"}
    ]
})

# NOR (NOT OR) --> 조건 중 하나라도 맞으면 제외
students.find({
    "$nor": [
        {"grade": "C"},
        {"age": {"$lt": 15}}
    ]
})

In [6]:
'''
개수 제한
'''

print(list(students.find().limit(2)))  # 2개만 가져오기

[{'_id': ObjectId('693563334a167e0bc926300f'), 'name': 'Alice', 'age': 22, 'grade': 'A', 'major': 'CS', 'score': 91}, {'_id': ObjectId('693563334a167e0bc9263010'), 'name': 'Bob', 'age': 17, 'grade': 'B', 'major': 'EE', 'score': 77}]


In [10]:
'''
정렬
'''

sorted_list = list(students.find().sort("age", -1))


for i in sorted_list:
    print(i)

{'_id': ObjectId('692d87ca1c54f1da6b6a932e'), 'name': 'Charlie', 'age': 24}
{'_id': ObjectId('692d87ca1c54f1da6b6a932d'), 'name': 'Bob', 'age': 23}
{'_id': ObjectId('692d87ca1c54f1da6b6a932f'), 'name': 'Jihoo', 'age': 13}


In [ ]:
'''
필요한 필드만 조회
'''

# 0: 제외, 1: 포함, 없음: 제외

selected_list = list(students.find(
    {},
    {'_id': 0}   
))

print(selected_list)


[{'name': 'Bob', 'age': 23}, {'name': 'Charlie', 'age': 24}, {'name': 'Jihoo', 'age': 13}]


2025-12-07 22:10:59,344| ERROR   | Could not establish connection from local ('127.0.0.1', 61644) to remote ('localhost', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2025-12-07 22:11:06,951| ERROR   | Could not establish connection from local ('127.0.0.1', 61644) to remote ('localhost', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2025-12-07 22:25:08,512| ERROR   | Could not establish connection from local ('127.0.0.1', 61644) to remote ('localhost', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2025-12-07 22:25:08,907| ERROR   | Socket exception: Connection reset by peer (54)
2025-12-07 22:25:13,909| ERROR   | Could not establish connection from local ('127.0.0.1', 61644) to remote ('localhost', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2025-12-07 22:25:18,476| ERROR   | Could not establish connection from local ('127.0.0.1', 61644) to remote ('l

In [ ]:
'''
문서 개수 세기
'''

students.count_documents({})
students.count_documents({"age": {"$gte": 23}})

2

In [ ]:
'''
문서 한 개 수정
'''

"""
1. filter (필수)
    업데이트할 문서를 찾기 위한 조건을 지정합
    MongoDB의 find 메서드와 동일한 형식의 조건을 사용
    예: {"name": "Alice"}는 name 필드가 "Alice"인 문서를 찾음

2. update (필수)
    문서를 어떻게 업데이트할지 지정
    $set, $inc, $unset 등 다양한 연산자를 사용 가능
    예: {"$set": {"age": 25}}는 age 필드를 25로 설정
    
3. options (선택)
    추가적인 옵션을 설정
    upsert: 조건에 맞는 문서가 없으면 새 문서를 삽입.
    기본값은 False
    예: {"upsert": True}
"""

students.update_one(
    {"name": "Alice"},
    {"$set": {"age": 25}},
    {'upsert': True}
)

In [ ]:
'''
문서 여러 개 수정
'''

"""
update_one의 여러 문서 수정 버전, 사용법 같음
"""
students.update_many({}, {"$inc": {"age": 1}})


In [ ]:
'''
문서 전체 교체
'''

"""
1. filter (필수)
    교체할 문서를 찾기 위한 조건을 지정
    MongoDB의 find 메서드와 동일한 형식의 조건을 사용
    예: {"name": "Bob"}는 name 필드가 "Bob"인 문서를 찾음
2. replacement (필수)
    기존 문서를 대체할 새 문서를 지정
    주의: replacement는 새 문서의 전체 내용을 포함해야 함
    예: {"name": "Bobby", "age": 21}는 새 문서로 교체함
"""

students.replace_one({"name": "Bob"}, {"name": "Bobby", "age": 21})

In [ ]:
'''
문서 한 개 삭제
'''

students.delete_one({"name": "Charlie"})

In [ ]:
'''
문서 여러 개 삭제
'''

students.delete_many({"age": {"$lt": 20}})